In [1]:
# Import the libraies
import numpy as np
from flask import Flask, request, render_template
import joblib
import pandas as pd


In [2]:
app = Flask(__name__)
model = joblib.load('models/model.pkl')

In [3]:
# Define the '/' root route to display the content from index.html
@app.route('/')
def home():
    return render_template('index.html')


In [4]:
# Define the '/predict' route to:
# - Get form data and convert them to float values
# - Convert form data to numpy array
# - Pass form data to model for prediction
from flask import render_template, request
import numpy as np

@app.route('/predict', methods=['POST'])
def predict():
    # Get form data and convert them to the correct data types
    no_of_dependents = int(request.form['no_of_dependents'])
    income_annum = float(request.form['income_annum'])
    loan_amount = float(request.form['loan_amount'])
    loan_term = int(request.form['loan_term'])
    cibil_score = int(request.form['cibil_score'])
    residential_assets_value = float(request.form['residential_assets_value'])
    commercial_assets_value = float(request.form['commercial_assets_value'])
    luxury_assets_value = float(request.form['luxury_assets_value'])
    bank_asset_value = float(request.form['bank_asset_value'])
    self_employed = 1 if request.form['self_employed'] == 'yes' else 0
    education = 1 if request.form['education'] == 'graduate' else 0

    # Convert cibil_score to a 1-dimensional array
    cibil_score_array = np.array([cibil_score])

    # Binning for cibil_score
    bins = [0, 600, 700, 800, 900]
    labels = ['Poor', 'Fair', 'Good', 'Excellent']
    cibil_score_category = pd.cut(cibil_score_array, bins=bins, labels=labels)

    # Calculate additional features based on your data preprocessing
    total_asset = residential_assets_value + commercial_assets_value + luxury_assets_value + bank_asset_value
    loan_to_income_ratio = loan_amount / income_annum

    # Create a numpy array with the input features
    features = np.array([
        no_of_dependents,
        income_annum,
        loan_amount,
        loan_term,
        cibil_score,
        residential_assets_value,
        commercial_assets_value,
        luxury_assets_value,
        bank_asset_value,
        total_asset,  # Add the total_asset feature
        loan_to_income_ratio,  # Add the loan_to_income_ratio feature
        self_employed,
        education,
        # Add the cibil_score_category features
        cibil_score_category[0] == 'Poor',
        cibil_score_category[0] == 'Fair',
        cibil_score_category[0] == 'Good',
        cibil_score_category[0] == 'Excellent'
    ]).reshape(1, -1)

    # Make prediction using the trained model
    prediction = model.predict(features)

   # Display user-friendly message based on the prediction
    if prediction[0]:
        message = "Your Loan is most likely to be approved.🏆"
    else:
        message = "Your Loan is most likely to be rejected.😓"
        image_url = None

    # Pass the message to the HTML template
    return render_template('index.html', loan_status=message)


In [ ]:
if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Dec/2023 12:40:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2023 12:40:29] "GET / HTTP/1.1" 200 -
/Users/swanhtetaung/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [07/Dec/2023 12:40:58] "POST /predict HTTP/1.1" 200 -
